ZERO TEST

In [113]:
# Import necessary libraries

# RDFlib
from rdflib import Graph, URIRef, Namespace
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import FOAF, NamespaceManager

# SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON

# pandas
import pandas as pd

# sparql_dataframe
import sparql_dataframe

# prettyprinter
from prettyprinter import pprint

# matplotlib
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns

# datatime
from datetime import datetime

# json
import json

# os
import os

# regular expressions
import re

from bokeh.io import output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
directory = '/Users/stefano_renzetti/Library/CloudStorage/Dropbox/DHDK/INFORMATION VISUALIZATION/_sources/FONDAZIONE-ZERI-RDF/dump-complete'

# Initialize an RDF graph
zeri_graph = Graph()

try:
    # Iterate through files in the directory, parse each file into a separate graph, and merge with the combined graph
    for filename in os.listdir(directory):
        if filename.endswith('.rdf'):  # Adjust the file extension if needed
            file_path = os.path.join(directory, filename)
            zeri_graph.parse(file_path, format='xml', strict=False)  # Allow parsing with invalid URIs
            #combined_graph += graph

    print("RDF data successfully loaded from the directory.")

    # Determine the number of triples in the graph
    number_of_triples = len(zeri_graph)
    
    print(f"The graph contains {number_of_triples} triples.")

except Exception as e:
    print("Failed to load RDF data from the directory:", e)

https://w3id.org/zericatalog/criterion/Author's choice does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Title printed on recto of photograph does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Author's choice does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Title printed on recto of photograph does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Author's choice does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Title printed on recto of photograph does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Author's choice does not look like a valid URI, trying to serialize this will break.
https://w3id.org/zericatalog/criterion/Title printed on recto of phot

RDF data successfully loaded from the directory.
The graph contains 11992953 triples.


In [4]:
# Define the SPARQL query
query = """
    SELECT DISTINCT ?class_uri
    WHERE {
        ?anything a ?class_uri .
    } LIMIT 10
"""

# Prepare the SPARQL query
query_obj = prepareQuery(query)

# Execute the query and retrieve the results
lists_classes = zeri_graph.query(query_obj)

# Uncomment the two following lines to see the list of classes
for row in lists_classes:
    print(row['class_uri'])

http://purl.org/emmedi/oaentry/OAEntry
http://www.cidoc-crm.org/cidoc-crm/E31_Document
http://www.cidoc-crm.org/cidoc-crm/E35_Title
http://www.cidoc-crm.org/cidoc-crm/E15_Identifier_Assignment
http://www.cidoc-crm.org/cidoc-crm/E42_Identifier
http://www.cidoc-crm.org/cidoc-crm/E55_Type
http://www.cidoc-crm.org/cidoc-crm/E40_Legal_Body
http://www.cidoc-crm.org/cidoc-crm/E65_Creation
http://www.cidoc-crm.org/cidoc-crm/E21_Person
http://purl.org/spar/fabio/MetadataDocument


In [5]:
# Get the namespace manager of the graph
namespace_manager = zeri_graph.namespace_manager

# Retrieve all prefixes used in the RDF graph
prefixes = {str(prefix): str(namespace_manager.store.namespace(prefix)) for prefix in namespace_manager.namespaces()}

# Display the prefixes and their corresponding URIs
for prefix, uri in prefixes.items():
    print(f"Prefix: {prefix}")

Prefix: ('brick', rdflib.term.URIRef('https://brickschema.org/schema/Brick#'))
Prefix: ('csvw', rdflib.term.URIRef('http://www.w3.org/ns/csvw#'))
Prefix: ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/'))
Prefix: ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#'))
Prefix: ('dcmitype', rdflib.term.URIRef('http://purl.org/dc/dcmitype/'))
Prefix: ('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms/'))
Prefix: ('dcam', rdflib.term.URIRef('http://purl.org/dc/dcam/'))
Prefix: ('doap', rdflib.term.URIRef('http://usefulinc.com/ns/doap#'))
Prefix: ('foaf', rdflib.term.URIRef('http://xmlns.com/foaf/0.1/'))
Prefix: ('geo', rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#'))
Prefix: ('odrl', rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/'))
Prefix: ('org', rdflib.term.URIRef('http://www.w3.org/ns/org#'))
Prefix: ('prof', rdflib.term.URIRef('http://www.w3.org/ns/dx/prof/'))
Prefix: ('prov', rdflib.term.URIRef('http://www.w3.org/ns/prov#'))
Prefix: ('qb', rdflib.te

## How many painters are presented in the catalog?

In [11]:
# Define the SPARQL query
query = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?artist ?artistLabel
    WHERE {
        ?artist pro:withRole oaentry:painter ;
                rdfs:label ?artistLabel .
        FILTER(LANG(?artistLabel) = 'en')
    }
"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:

        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results, columns=results.vars)

        print("Number of painters:", len(df))

    else:
        
        print("No painters founded")

except Exception as e:
    
    print("An error occurred:", e)

df.info()
df

Number of painters: 4465
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist       4465 non-null   object
 1   artistLabel  4465 non-null   object
dtypes: object(2)
memory usage: 69.9+ KB


,artist,artistLabel
0,https://w3id.org/zericatalog/role/painter/bemb...,"Bembo Bonifacio, painter"
1,https://w3id.org/zericatalog/role/painter/pret...,"Preti Gregorio, painter"
2,https://w3id.org/zericatalog/role/painter/tucc...,"Tuccari Giovanni, painter"
3,https://w3id.org/zericatalog/role/painter/euse...,"Eusebio da San Giorgio, painter"
4,https://w3id.org/zericatalog/role/painter/giac...,"Giacomo da Riva, painter"
...,...,...
4460,https://w3id.org/zericatalog/role/painter/erri...,"Erri Bartolomeo degli, painter"
4461,https://w3id.org/zericatalog/role/painter/capr...,"Caprioli Domenico, painter"
4462,https://w3id.org/zericatalog/role/painter/pseu...,"Pseudo Scacco, painter"
4463,https://w3id.org/zericatalog/role/painter/stro...,"Stroifi Ermanno, painter"


## How many titled artworks are presented in the catalog?

In [12]:
# Define the SPARQL query
query_artwork = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>

SELECT DISTINCT ?artwork ?artworkTitleLabel
WHERE { 
    ?photographs frbr:subject ?artwork ;
                 fabio:hasSubjectTerm ?photoSubject ;
                 crm:P102_has_title ?artworkTitle .
                 
    ?artworkTitle rdfs:label ?artworkTitleLabel .
    FILTER(LANG(?artworkTitleLabel) = 'en')
}

"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query_artwork)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:

        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results, columns=results.vars)

        print("Number of artworks:", len(df))

    else:
        
        print("No artworks founded")

except Exception as e:
    print("An error occurred:", e)

df.info()
df

Number of artworks: 25711
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25711 entries, 0 to 25710
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artwork            25711 non-null  object
 1   artworkTitleLabel  25711 non-null  object
dtypes: object(2)
memory usage: 401.9+ KB


,artwork,artworkTitleLabel
0,https://w3id.org/zericatalog/artwork/42605/item,"Attributed title ""Negretti Iacopo - sec. XVI -..."
1,https://w3id.org/zericatalog/artwork/16567/item,"Proper title ""Roma. Cappella Sistina. Figura a..."
2,https://w3id.org/zericatalog/artwork/27111/item,"Attributed title ""Giovanni di Pietro - sec. XV..."
3,https://w3id.org/zericatalog/artwork/37625/item,"Attributed title ""Anonimo napoletano - sec. XV..."
4,https://w3id.org/zericatalog/artwork/41044/item,"Proper title ""A Portrait of a Gentleman wearin..."
...,...,...
25706,https://w3id.org/zericatalog/artwork/29553/item,"Proper title ""Spoleto - S. Maria delle Grazie ..."
25707,https://w3id.org/zericatalog/artwork/48045/item,"Proper title ""Lorenzo Lotto, Italian, Venetian..."
25708,https://w3id.org/zericatalog/artwork/27093/item,"Attributed title ""Giovanni di Pietro - sec. XV..."
25709,https://w3id.org/zericatalog/artwork/36898/item,"Attributed title ""Foschi Pier Francesco - sec...."


## What are all the subjects portrayed by each photograph?

In [13]:
# Define the SPARQL query
query = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX : <http://www.essepuntato.it/2014/03/fentry-abox/>
    PREFIX fentry: <http://www.essepuntato.it/2014/03/fentry/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>

    SELECT DISTINCT ?photo ?subject ?subjectLabel
    WHERE { 
    	?photo a fentry:Photograph ;
        	   frbr:subject ?subject .
            
        ?subject rdfs:label ?subjectLabel .
        FILTER(LANG(?subjectLabel) = 'en')        
    }
"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:

        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results, columns=results.vars)

        print("Number of pictured subjects", len(df))

    else:
        
        print("No pictured subjects founded")

except Exception as e:
    print("An error occurred:", e)

df.info()
df

Number of pictured subjects 31147
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31147 entries, 0 to 31146
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   photo         31147 non-null  object
 1   subject       31147 non-null  object
 2   subjectLabel  31147 non-null  object
dtypes: object(3)
memory usage: 730.1+ KB


,photo,subject,subjectLabel
0,https://w3id.org/zericatalog/photo/85413,https://w3id.org/zericatalog/artwork/42605/item,"Artwork ""Negretti Iacopo , Sacra Famiglia con ..."
1,https://w3id.org/zericatalog/photo/38192,https://w3id.org/zericatalog/artwork/16567/item,"Artwork ""Buonarroti Michelangelo , Separazione..."
2,https://w3id.org/zericatalog/photo/59578,https://w3id.org/zericatalog/artwork/27111/item,"Artwork ""Giovanni di Pietro , Madonna con Bamb..."
3,https://w3id.org/zericatalog/photo/77685,https://w3id.org/zericatalog/artwork/37625/item,"Artwork ""Anonimo napoletano sec. XVII , Gruppo..."
4,https://w3id.org/zericatalog/photo/83062,https://w3id.org/zericatalog/artwork/41044/item,"Artwork ""Passarotti Bartolomeo , Autoritratto ..."
...,...,...,...
31142,https://w3id.org/zericatalog/photo/79869,https://w3id.org/zericatalog/artwork/39092/item,"Artwork ""Vasari Giorgio , Ritratto di Pierfran..."
31143,https://w3id.org/zericatalog/photo/77844,https://w3id.org/zericatalog/artwork/37718/item,"Artwork ""Tosini Michele , Figura femminile"" - ..."
31144,https://w3id.org/zericatalog/photo/59561,https://w3id.org/zericatalog/artwork/27093/item,"Artwork ""Giovanni di Pietro , Madonna con Bamb..."
31145,https://w3id.org/zericatalog/photo/76582,https://w3id.org/zericatalog/artwork/36898/item,"Artwork ""Foschi Pier Francesco , Cristo in pie..."


In [49]:

# Define the SPARQL query
query = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX : <http://www.essepuntato.it/2014/03/fentry-abox/>
    PREFIX fentry: <http://www.essepuntato.it/2014/03/fentry/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>
    PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX emmedi: <http://purl.org/emmedi/>
    PREFIX hico: <http://purl.org/emmedi/hico/>
    PREFIX terms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?artisticArtwork ?manifestation ?p ?analogItem ?o
WHERE {
    #<https://w3id.org/zericatalog/artwork/18631/item> a ?itemIsA .

    ?artisticArtwork a fabio:ArtisticWork;
                     fabio:hasManifestation ?manifestation.
    #?oaEntry a oaentry:OAEntry .
    ?analogItem a fabio:AnalogItem .
    
    #?oaEntry oaentry:describes ?o .
    ?manifestation ?p ?analogItem .

   #?s frbr:exemplar ?o.

} LIMIT 10

"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:
        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results.bindings)


except Exception as e:
    print("An error occurred:", e)

df.head(60)


,analogItem,artisticArtwork,manifestation,p
0,https://w3id.org/zericatalog/artwork/18157/item,https://w3id.org/zericatalog/artwork/18157,https://w3id.org/zericatalog/artwork/18157/man...,http://purl.org/vocab/frbr/core#exemplar
1,https://w3id.org/zericatalog/artwork/42605/item,https://w3id.org/zericatalog/artwork/42605,https://w3id.org/zericatalog/artwork/42605/man...,http://purl.org/vocab/frbr/core#exemplar
2,https://w3id.org/zericatalog/artwork/16567/item,https://w3id.org/zericatalog/artwork/16567,https://w3id.org/zericatalog/artwork/16567/man...,http://purl.org/vocab/frbr/core#exemplar
3,https://w3id.org/zericatalog/artwork/38287/item,https://w3id.org/zericatalog/artwork/38287,https://w3id.org/zericatalog/artwork/38287/man...,http://purl.org/vocab/frbr/core#exemplar
4,https://w3id.org/zericatalog/artwork/27111/item,https://w3id.org/zericatalog/artwork/27111,https://w3id.org/zericatalog/artwork/27111/man...,http://purl.org/vocab/frbr/core#exemplar
5,https://w3id.org/zericatalog/artwork/41022/item,https://w3id.org/zericatalog/artwork/41022,https://w3id.org/zericatalog/artwork/41022/man...,http://purl.org/vocab/frbr/core#exemplar
6,https://w3id.org/zericatalog/artwork/46995/item,https://w3id.org/zericatalog/artwork/46995,https://w3id.org/zericatalog/artwork/46995/man...,http://purl.org/vocab/frbr/core#exemplar
7,https://w3id.org/zericatalog/artwork/37625/item,https://w3id.org/zericatalog/artwork/37625,https://w3id.org/zericatalog/artwork/37625/man...,http://purl.org/vocab/frbr/core#exemplar
8,https://w3id.org/zericatalog/artwork/41044/item,https://w3id.org/zericatalog/artwork/41044,https://w3id.org/zericatalog/artwork/41044/man...,http://purl.org/vocab/frbr/core#exemplar
9,https://w3id.org/zericatalog/artwork/47453/item,https://w3id.org/zericatalog/artwork/47453,https://w3id.org/zericatalog/artwork/47453/man...,http://purl.org/vocab/frbr/core#exemplar


## Let's inspect frther the level of Work-Expression-Manifestation-Item relationship in FaBiO ontology.
- ***oaentry:OAEntry***
It is a document containing metadata about a work of art, i.e. a fabio:ArtisticWork, and about the concrete object portrayed by the work of art.
- ***crm:E65_Creation***
Is refered to the creation of the OA Entry (document).
- ***fabio:MetadataDocument***
It just express the content of the OA Entry.
- ***fabio:ArtisticWork***
From OA Entry documentation:
> we decided to represent a work of art as an FRBR Work when describing its essence, as an FRBR Manifestation when providing information about its physical features that may vary over time, and as an FRBR Item when dealing with information about legal aspects and its location. Modelling works of art in these terms means taking into account established considerations in the arts domain
It describes any work regarded as art in its widest sense, including works from literature and music, visual art, etc.

It contains the type of the work (i.e. painting) and the subject depicted
- ***crm:E65_Creation***
It is the Manifestation level. It describes the act of the creation and we can find the time span of the creation (crm:P4_has_time_span), the artist/author (crm:P14_carried_out_by and also the artist's group attribution (crm:P107i_is_current_or_former_member_of)
- ***fabio:AnalogManifestation***
It is the Item level. We can find description of the material for example (crm:P45_consists_of -> crm:E57_Material).
- ***fabio:AnalogItem***
We find indication about the place where the Item is located (crm:P74_has_current_or_former_residence).

In [95]:
#TO UNDERSTOOD HOW TO CONNECT ARTWORK AND ARTIST!!!

endpoint_wikidata = "https://query.wikidata.org/sparql"

# Define the SPARQL query
query = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX : <http://www.essepuntato.it/2014/03/fentry-abox/>
    PREFIX fentry: <http://www.essepuntato.it/2014/03/fentry/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>
    PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX emmedi: <http://purl.org/emmedi/>
    PREFIX hico: <http://purl.org/emmedi/hico/>
    PREFIX terms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX psv: <http://www.wikidata.org/prop/statement/value/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wds: <http://www.wikidata.org/entity/statement/>
    PREFIX wdv: <http://www.wikidata.org/value/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?oaTitleLabel ?artwork ?artworkTypeLabel ?artistLabel ?startDate ?keeperLabel ?locationLabel
WHERE {
    ?oaEntry oaentry:describes ?artwork ;
             crm:P102_has_title ?oaTitle .
    ?oaTitle rdfs:label ?oaTitleLabel .
    FILTER(LANG(?oaTitleLabel) = 'en')

    ?artwork a fabio:ArtisticWork  ;
             #fabio:hasSubjectTerm ?subject ;
             crm:P94i_was_created_by ?creation ;
             #crm:P2_has_type ?artworkType ;
             fabio:hasManifestation ?manifestation .
                 
    #?artworkType rdfs:label ?artworkTypeLabel .
    #FILTER(LANG(?artworkTypeLabel) = 'en')

    #?subject rdfs:label ?subjectLabel .
    #FILTER(LANG(?subjectLabel) = 'it')

    ?creation crm:P4_has_time_span ?timeSpan ;
              crm:P14_carried_out_by ?artist .
    ?timeSpan ti:hasIntervalStartDate ?startDate .

    FILTER(
    xsd:dateTime(?startDate) >= xsd:dateTime("1400-01-01T00:00:00Z") &&
    xsd:dateTime(?startDate) <= xsd:dateTime("1600-12-31T23:59:59Z")
    )

    ?artist rdfs:label ?artistLabel .

    ?manifestation frbr:exemplar ?analogItem .

    ?analogItem crm:P50_has_current_keeper ?keeper .
    OPTIONAL {?keeper rdfs:label ?keeperLabel}

    ?keeper crm:P74_has_current_or_former_residence ?location .
    ?location rdfs:label ?locationLabel .

}
ORDER BY ?startDate

"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:

        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results, columns=results.vars)

        #df = df.set_axis(['oaTitleLabel', 'artwork', 'subjectLabel', 'artworkTypeLabel', 'artistLabel', 'startDate', 'keeperLabel', 'locationLabel'], axis='columns')
        
        # Remove both parentheses and their content from 'locationLabel'
        #df['locationLabel'] = df['locationLabel'].str.replace(r'\s*\(.*?\)', '', regex=True)

except Exception as e:
    print("An error occurred:", e)

filtered_df = df.drop_duplicates()

filtered_df.info()
filtered_df.head(50)


<class 'pandas.core.frame.DataFrame'>
Index: 85656 entries, 0 to 147861
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   oaTitleLabel      85656 non-null  object
 1   artwork           85656 non-null  object
 2   artworkTypeLabel  85656 non-null  object
 3   artistLabel       85656 non-null  object
 4   startDate         85656 non-null  object
 5   keeperLabel       85656 non-null  object
 6   locationLabel     85656 non-null  object
dtypes: object(7)
memory usage: 5.2+ MB


,oaTitleLabel,artwork,artworkTypeLabel,artistLabel,startDate,keeperLabel,locationLabel
0,"Anonimo umbro sec. XV , San Nicola di Bari",https://w3id.org/zericatalog/artwork/29205,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Pinacoteca Comunale,Assisi
1,"Anonimo umbro sec. XV , San Nicola di Bari",https://w3id.org/zericatalog/artwork/29205,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Pinacoteca Comunale,Assisi
2,"Anonimo umbro sec. XV , Sant'Antonio Abate",https://w3id.org/zericatalog/artwork/29470,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
3,"Anonimo umbro sec. XV , Sant'Antonio Abate",https://w3id.org/zericatalog/artwork/29471,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
4,"Anonimo umbro sec. XV , Santi",https://w3id.org/zericatalog/artwork/29473,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
5,"Anonimo umbro sec. XV , Cristo giudice in tron...",https://w3id.org/zericatalog/artwork/29472,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
6,"Anonimo umbro sec. XV , Santo",https://w3id.org/zericatalog/artwork/29489,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
7,"Anonimo umbro sec. XV , Persona a tre teste co...",https://w3id.org/zericatalog/artwork/29461,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
8,"Anonimo umbro sec. XV , San Giorgio e il drago",https://w3id.org/zericatalog/artwork/29449,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
9,"Anonimo umbro sec. XV , Sant'Antonio Abate",https://w3id.org/zericatalog/artwork/29448,fresco,Anonimo umbro sec. XV,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni


In [213]:
endpoint = "https://query.wikidata.org/sparql"

# query female portraits and their location in Wikidata
query = """ 
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX fabio: <http://purl.org/spar/fabio/>
PREFIX frbr: <http://purl.org/vocab/frbr/core#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX psv: <http://www.wikidata.org/prop/statement/value/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wds: <http://www.wikidata.org/entity/statement/>
PREFIX wdv: <http://www.wikidata.org/value/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?artwork ?keeper ?keeperLabel ?lat ?long WHERE { 
  ?s fabio:hasSubjectTerm <https://w3id.org/zericatalog/subject/santantonio-abate> ; 
     fabio:hasManifestation ?manif .
  ?manif frbr:exemplar ?item . 
  
  ?item crm:P50_has_current_keeper ?keeper ; rdfs:label ?artwork.

  ?keeper crm:P74_has_current_or_former_residence ?location .
  OPTIONAL {?keeper rdfs:label ?keeperLabel}
  ?location owl:sameAs ?wdlocation .
  FILTER(LANG(?artwork) = "" || LANGMATCHES(LANG(?artwork), "en"))
  FILTER(LANG(?keeperLabel) = "" || LANGMATCHES(LANG(?keeperLabel), "en"))
  FILTER(contains (str(?wdlocation), "wikidata") )
  
  SERVICE <https://query.wikidata.org/sparql> {
    ?wdlocation p:P625 ?coordinate .
 ?coordinate psv:P625 ?coordinate_node.
 ?coordinate_node wikibase:geoLongitude ?lon.
 ?coordinate_node wikibase:geoLatitude ?lat. 
  }
} LIMIT 10


"""
# use sparql dataframe to transform the SPARQL Json directly into a dataframe
df = sparql_dataframe.get(endpoint, q, post=True)
df.head()

# use sparql dataframe to transform the SPARQL Json directly into a dataframe
#df = sparql_dataframe.get(zeri_graph, q, post=True)
#df.head()
#df.info()





,artwork,keeper,keeperLabel,lat,long


QUESTO LO TENIAMO PER QUANDO DESCRIVO LE PERSONE PERCHÈ QUANDO HA PIÙ DI UNA SCUOLA CREA PIÙ RECORDS.



In [ ]:
    ?analogItem a crm:E84_Information_Carrier ;
                crm:P74_has_current_or_former_residence ?location .
                         


In [ ]:

# Museums in Barcelona with coordinates
SELECT ?item ?itemLabel ?coord ?lon ?lat
WHERE
{
 ?item wdt:P131 wd:Q1492.   # in the administrative territory of Barcelona
 ?item wdt:P31 wd:Q33506.   # is a museum
 ?item p:P625 ?coordinate.
 ?coordinate ps:P625 ?coord.
 ?coordinate psv:P625 ?coordinate_node.
 ?coordinate_node wikibase:geoLongitude ?lon.
 ?coordinate_node wikibase:geoLatitude ?lat.  
 SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
}

In [204]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Below we SELECT both the hot sauce items & their labels
# in the WHERE clause we specify that we want labels as well as items
sparql.setQuery("""
SELECT ?item ?itemLabel

WHERE {
  ?item wdt:P279 wd:Q522171.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)

{'head': {'vars': ['item', 'itemLabel']}, 'results': {'bindings': [{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q249114'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'salsa'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q335016'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Tabasco sauce'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q360459'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Adobo'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q460439'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': "Blair's 16 Million Reserve"}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q736782'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Llajua'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q966327'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'harissa'}}, {'item': {'ty

In [ ]:
subject

In [ ]:
# PROVA COMPLETA CHE NON FUNZIONA. USARE PARTI DIVERSE PER TEST


PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>
    PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>

SELECT DISTINCT ?oaEntry ?artworkTitleLabel ?artistLabel ?startDate ?artisticGroupLabel ?artworkTypeLabel
WHERE { 
    ?oaEntry a oaentry:OAEntry .

    ?artwork a fabio:ArtisticWork ;
             crm:P2_has_type ?artworkType .
    ?artworkType rdfs:label ?artworkTypeLabel .
    FILTER(LANG(?artworkTypeLabel) = 'en')
             
    ?manifestation a crm:E65_Creation ;
                   crm:P4_has_time_span ?timeSpan ;
                   crm:P14_carried_out_by ?artist.
    ?timeSpan ti:hasIntervalStartDate ?startDate .
    ?artist rdfs:label ?artistLabel ;
            crm:P107i_is_current_or_former_member_of ?artisticGroup.
    ?artisticGroup rdfs:label ?artisticGroupLabel
    FILTER(LANG(?artisticGroupLabel) = 'en')

    ?analogManifestation a fabio:AnalogManifestation ;
                         rdfs:label ?artworkTitleLabel ;
                         crm:P45_consists_of ?material.
    FILTER(LANG(?artworkTitleLabel) = 'en')

    ?material rdfs:label ?materialLabel .
    FILTER(LANG(?materialLabel) = 'en')
}

In [148]:
#TO UNDERSTOOD HOW TO CONNECT ARTWORK AND ARTIST!!!

endpoint_wikidata = "https://query.wikidata.org/sparql"

# Define the SPARQL query
query = """
    PREFIX zeri1: <http://www.w3.org/ns/prov-o#>
    PREFIX pro: <http://purl.org/spar/pro/>
    PREFIX : <http://www.essepuntato.it/2014/03/fentry-abox/>
    PREFIX fentry: <http://www.essepuntato.it/2014/03/fentry/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX oaentry: <http://purl.org/emmedi/oaentry/>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX frbr: <http://purl.org/vocab/frbr/core#>
    PREFIX prov: <http://www.w3.org/ns/prov#>
    PREFIX fabio: <http://purl.org/spar/fabio/>
    PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX emmedi: <http://purl.org/emmedi/>
    PREFIX hico: <http://purl.org/emmedi/hico/>
    PREFIX terms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX psv: <http://www.wikidata.org/prop/statement/value/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wds: <http://www.wikidata.org/entity/statement/>
    PREFIX wdv: <http://www.wikidata.org/value/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX p: <http://www.wikidata.org/prop/>
    PREFIX ps: <http://www.wikidata.org/prop/statement/>
    PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?artwork ?artworkLabel ?artworkTypeLabel ?startDate ?keeperLabel ?locationLabel
WHERE {

    ?artwork a fabio:ArtisticWork  ;
             rdfs:label ?artworkLabel ;
             crm:P94i_was_created_by ?creation ;
             fabio:hasManifestation ?manifestation ;
             crm:P2_has_type ?artworkType .
                 
    ?artworkType rdfs:label ?artworkTypeLabel .
    FILTER(LANG(?artworkTypeLabel) = 'en')
                 
    ?creation crm:P4_has_time_span ?timeSpan .
    ?timeSpan ti:hasIntervalStartDate ?startDate .

    ?manifestation frbr:exemplar ?analogItem .

    ?analogItem crm:P50_has_current_keeper ?keeper .
    ?keeper rdfs:label ?keeperLabel .

    ?keeper crm:P74_has_current_or_former_residence ?location .
    ?location rdfs:label ?locationLabel .
    
    FILTER(
    xsd:dateTime(?startDate) >= xsd:dateTime("1400-01-01T00:00:00Z") &&
    xsd:dateTime(?startDate) <= xsd:dateTime("1600-12-31T23:59:59Z")
    )

}
ORDER BY ?startDate

"""

try:
    # Prepare the SPARQL query
    query_obj = prepareQuery(query)
    
    # Execute the query and retrieve the results
    results = zeri_graph.query(query_obj)

    if results:

        # Convert SPARQL results to a pandas DataFrame
        df = pd.DataFrame(results, columns=results.vars)

        df = df.set_axis(['artwork','artworkLabel', 'artworkTypeLabel', 'startDate', 'keeperLabel', 'locationLabel'], axis='columns')
                
        # Modify the regex pattern to drop two capital letters at the end of 'locationLabel' with the last space
        df['locationLabel'] = df['locationLabel'].str.replace(r'\s+[A-Z]{2}$', '', regex=True)

except Exception as e:
    print("An error occurred:", e)

filtered_df = df.drop_duplicates(['artwork'])

# Reindex the DataFrame to create a continuous index sequence
filtered_df = filtered_df.reset_index(drop=True)

df.info()

filtered_df.info()
filtered_df.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264318 entries, 0 to 264317
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   artwork           264318 non-null  object
 1   artworkLabel      264318 non-null  object
 2   artworkTypeLabel  264318 non-null  object
 3   startDate         264318 non-null  object
 4   keeperLabel       264318 non-null  object
 5   locationLabel     264318 non-null  object
dtypes: object(6)
memory usage: 12.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15944 entries, 0 to 15943
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artwork           15944 non-null  object
 1   artworkLabel      15944 non-null  object
 2   artworkTypeLabel  15944 non-null  object
 3   startDate         15944 non-null  object
 4   keeperLabel       15944 non-null  object
 5   locationLabel     15944 non-null  obje

,artwork,artworkLabel,artworkTypeLabel,startDate,keeperLabel,locationLabel
0,https://w3id.org/zericatalog/artwork/29205,"Artwork ""Anonimo umbro sec. XV , San Nicola di...",fresco,1400-01-01T00:00:00Z,Pinacoteca Comunale,Assisi
1,https://w3id.org/zericatalog/artwork/29470,"Artwork ""Anonimo umbro sec. XV , Sant'Antonio ...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
2,https://w3id.org/zericatalog/artwork/29471,"Artwork ""Anonimo umbro sec. XV , Sant'Antonio ...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
3,https://w3id.org/zericatalog/artwork/29473,"Artwork ""Anonimo umbro sec. XV , Santi"" , (fra...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
4,https://w3id.org/zericatalog/artwork/29472,"Artwork ""Anonimo umbro sec. XV , Cristo giudic...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
5,https://w3id.org/zericatalog/artwork/29489,"Artwork ""Anonimo umbro sec. XV , Santo"" , (fra...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Agostino,Narni
6,https://w3id.org/zericatalog/artwork/29461,"Artwork ""Anonimo umbro sec. XV , Persona a tre...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
7,https://w3id.org/zericatalog/artwork/29449,"Artwork ""Anonimo umbro sec. XV , San Giorgio e...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
8,https://w3id.org/zericatalog/artwork/29448,"Artwork ""Anonimo umbro sec. XV , Sant'Antonio ...",fresco,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni
9,https://w3id.org/zericatalog/artwork/29451,"Artwork ""Anonimo umbro sec. XV , Testa d'uomo ...",mural drawing,1400-01-01T00:00:00Z,Chiesa di S. Francesco,Narni


In [136]:
#QUERY ALL THE ITALIAN MUNICIPALITIES AND THEIR COORDINATES -!!!THEY NEED TO BE CONVERTED

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Below we SELECT both the hot sauce items & their labels
# in the WHERE clause we specify that we want labels as well as items
sparql.setQuery("""

    
SELECT ?city ?cityLabel ?lat ?long 

WHERE {
  ?city wdt:P31 wd:Q747074;
        p:P625 ?coords_stmt .
  ?coords_stmt ps:P625 ?coords;
         psv:P625 [
           wikibase:geoLatitude ?lat;
           wikibase:geoLongitude ?long ] .
     SERVICE wikibase:label {
     bd:serviceParam wikibase:language "it" .
   }

}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Formatting the JSON output with indentation for better readability
formatted_results = json.dumps(results, indent=4)

#print(formatted_results)

In [138]:
# Convert the JSON string to a Python dictionary
data = json.loads(formatted_results)

# Transforming the structure
transformed_data = []
for item in data['results']['bindings']:
    lat = float(item['lat']['value'])
    long = float(item['long']['value'])

    transformed_item = {
        'CITY': item['city']['value'],
        'CITY_LABEL': item['cityLabel']['value'],
        'LAT': lat,
        'LONG': long,
        'GEOCODE': [lat, long]  # Creating a list of latitude and longitude
    }
    transformed_data.append(transformed_item)

# Displaying the transformed data
#print(transformed_data)

In [149]:
import json

# Your transformed_data (JSON) and df (DataFrame) are assumed to be available here.

# Create a dictionary to map CITY_LABEL to GEOCODE
city_to_geocode = {item['CITY_LABEL']: item['GEOCODE'] for item in transformed_data}

# Initialize a list to store the new JSON data
new_json_data = []

# Iterate through DataFrame rows
for index, row in filtered_df.iterrows():
    artwork = row['artwork']
    artworkTypeLabel = row['artworkTypeLabel']
    startDate = row['startDate']
    keeperLabel = row['keeperLabel']
    locationLabel = row['locationLabel']

    # Check if the locationLabel exists in city_to_geocode mapping
    if locationLabel in city_to_geocode:
        # Retrieve the GEOCODE (latitude and longitude) for the matching locationLabel
        geocode = city_to_geocode[locationLabel]

        # Create a new JSON structure using DataFrame values and matched geocode
        new_item = {
            'artwork': artwork,
            'artworkTypeLabel': artworkTypeLabel,
            'startDate': startDate,
            'keeperLabel': keeperLabel,
            'locationLabel': locationLabel,
            'GEOCODE': geocode
        }
        new_json_data.append(new_item)

# Convert the new JSON data to a JSON-formatted string
new_json_string = json.dumps(new_json_data, indent=4)


# Save the new file

# Specify the folder path to save the JSON file
folder_path = "_datasets/Clean/D1"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the filename for the JSON file
filename = "artwork_location.json"

# Generate the file path
file_path = os.path.join(folder_path, filename)

# Save the resident Italian population JSON to the file
with open(file_path, "w") as file:
    file.write(new_json_string)

print(f"JSON data saved to: {file_path}")

JSON data saved to: _datasets/Clean/D1/artwork_location.json
